# Examples of using fuction in "data_description.py"

In [1]:
import pandas as pd
import numpy as np
from data_description import *

# Creation an exmples data frame

In [24]:
frame_size = 100

data = pd.DataFrame({
    'dates_column' : np.datetime64('2017-01-01') + np.random.choice(np.arange(0, 60), frame_size),
    'object_column' : np.random.choice(['lev1', 'lev2', 'lev3'], frame_size),
    'category_column' : pd.Series(np.random.choice(['lev1', 'lev2', 'lev3'], frame_size), dtype = "category"),
    'bool_column' : np.random.choice([True, False], frame_size),
    'float_column' : np.random.normal(5, 1, frame_size),
    'integer_column' : np.random.normal(5, 1, frame_size).astype('int64')
})

for col_name in data.columns:
    data.loc[data.sample(20).index, col_name] = np.NaN

# Testing of fucntion which describes columns

## Testing of "get_col_av_values" function

In [25]:
data.apply(get_col_av_values)

dates_column                     [01.01.2017;01.03.2017]
object_column                      lev1, lev2, lev3, nan
category_column                    lev2, lev3, lev1, nan
bool_column                             False, nan, True
float_column       [2.772175016001849;7.262010316752875]
integer_column                                 [1.0;7.0]
dtype: object

## Testing of "get_col_obj_count" function

In [26]:
data.apply(get_col_obj_count)

dates_column       -
object_column      4
category_column    4
bool_column        3
float_column       -
integer_column     -
dtype: object

## Testing of "get_columns_desription" function

In [27]:
get_columns_desription(data)

,Data type,Range,Values number,NA count
dates_column,datetime64[ns],[01.01.2017;01.03.2017],-,20
object_column,object,"lev1, lev2, lev3, nan",4,20
category_column,category,"lev2, lev3, lev1, nan",4,20
bool_column,object,"False, nan, True",3,20
float_column,float64,[2.772175016001849;7.262010316752875],-,20
integer_column,float64,[1.0;7.0],-,20
